In [19]:
from pyspark.sql import types as T
from pyspark.sql import functions as F 
# from pyspark.sql.types import IntegerType
# from pyspark.sql.functions import first

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Table 1A+1B+2 
registrations_raw = spark.sql("""
    select 
        UID.id as user_id
        , to_date(dcreated) as reg_date
        , passwordset
        , subscriptionid
        , to_date(subscriptiondate) as sub_date
        , isactive
        , manualunsubscribe
        , to_date(logdatetime) as log_date
        , logaction
        , count(logdatetime) as n_events
    from stepstonede.userlogin UID
    left join stepstonede.subscription SUB
    on UID.id = SUB.userlogin_id
    left join stepstonede.logmystst LOG
    on 
        UID.id = LOG.userloginid 
        and logaction in ("MyStSt,AccountValidation","AUM,AccountValidation", "IJM, subscription", "SIM_LastOpen", "IJM, autodeactivate", "IJM, autoactivate", 
            "IJM, Manual Deactivate", "IJM, activation", "IJM, unsubscription")
    where 
        dcreated >= '2020-01-01' 
    group by
        UID.id 
        , reg_date
        , passwordset
        , subscriptionid
        , sub_date
        , isactive
        , manualunsubscribe
        , log_date
        , logaction
    order by reg_date, log_date""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Table 2: validated accounts
registration_df = registrations_raw.select("user_id").distinct().\
    join(registrations_raw.select("user_id", "reg_date", "logaction", "log_date"). \
        filter(F.col("logaction").isin(["MyStSt,AccountValidation","AUM,AccountValidation"])). \
        groupBy("user_id", "reg_date"). \
        agg(F.min("log_date").alias("AccountValidation")), "user_id", "left")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Table2A: Validated accounts & considered_dates
# create DF with all dates from first day to last day per user
# user_df =
considered_date_df = registrations_raw.select("user_id", "reg_date"). \
    distinct(). \
    withColumn("first_day", F.to_date(F.col("reg_date"))). \
    withColumn("last_day", F.to_date(F.lit(registrations_raw.select(F.max(F.col("reg_date")).alias("max_date")).collect()[0][0]))). \
    withColumn("diffDays",  F.datediff("last_day", "first_day")). \
    withColumn("repeat", F.expr("split(repeat(',', diffDays), ',')")).\
    select("*", F.posexplode("repeat").alias("daycount", "val")).\
    drop("repeat", "val", "diffDays").\
    withColumn("considered_date", F.expr("date_add(first_day, daycount)")).\
    select("user_id", "considered_date")

registration_df.join(considered_date_df, "user_id", "left") # this is Table2A

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Table 2B (just started this on our call)
registrations_raw. \
    filter(F.col("logaction").isin(["IJM, subscription"])). \
    select("user_id", "log_date", "n_events").show(10)
